# Exploring urban data with pandas

**pandas** is a Python library for analyzing and organizing tabular data. It's probably the most common library for working with both big and small datasets in Python, and is the basis for working with more analytical packages (e.g. scikit learn) and analyzing geographic data (e.g. geopandas)

This notebook provides an intro to pandas for analyzing urban data. We'll be learning the following
- how to load and save datasets into pandas DataFrames
- generating new columns
- filtering and subsetting DataFrames
- computing descriptive statistics
- aggregating and summarizing data by groups

In [27]:
import pandas as pd

Let's first take a look at a small dataset, Canadian municipalities and their population in 2021 and 2016, based on Census data. In Statistics Canada lingo, these are called [Census Subdivisions](https://www12.statcan.gc.ca/census-recensement/2021/ref/dict/az/Definition-eng.cfm?ID=geo012). This dataset only includes municipalities with a population greater than 25,000 in 2021.

The main method for loading csv data is to use the [read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) function, but pandas can also read many other [data formats](https://pandas.pydata.org/pandas-docs/stable/reference/io.html).

In [40]:
df = pd.read_csv("cities.csv")

Great! Now our data should be loaded to a DataFrame called `df`. We can print this and explore. Adding the function `.head()` or `.tail()` prints the top or bottom rows of the DataFrame. i.e. the following prints the top 10 rows in the DataFrame

In [41]:
df.head(6)

,Name,Prov/terr,"Population, 2021","Population, 2016"
0,Abbotsford,B.C.,153524,141397
1,Airdrie,Alta.,74100,61581
2,Ajax,Ont.,126666,119677
3,Alma,Que.,30331,30771
4,Aurora,Ont.,62057,55445
5,Barrie,Ont.,147829,141434


pandas has a number of functions for manipulating DataFrames like this.

For example, it looks like the data are are sorted by their name, but what if we wanted to sort by `Population, 2021`? We can use the [sort_values](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html?highlight=sort_values#pandas.DataFrame.sort_values) function.

The default is to sort in ascending order, so we set this to be `False` (i.e. descending) so the most populous cities are at the top.

In [43]:
df.sort_values('Population, 2021', ascending = False).head(6)

,Name,Prov/terr,"Population, 2021","Population, 2016"
158,Toronto,Ont.,2794356,2731571
89,Montréal,Que.,1762949,1704694
19,Calgary,Alta.,1306784,1239220
106,Ottawa,Ont.,1017449,934243
42,Edmonton,Alta.,1010899,933088
175,Winnipeg,Man.,749607,705244


We can also easily filter the DataFrame. Let's show only municipalities in New Brunswick. 

In [54]:
df.loc[df["Prov/terr"] == "N.B."]

,Name,Prov/terr,"Population, 2021","Population, 2016"
38,Dieppe,N.B.,28114,25384
45,Fredericton,N.B.,63116,58721
88,Moncton,N.B.,79470,71889
129,Saint John,N.B.,69895,67575


Looks like their are only these 4! You can try filtering for other provinces. If you want to get a list of all the provinces in the dataset, we can run the `unique()` function on the series.

In [55]:
df["Prov/terr"].unique()

array(['B.C.', 'Alta.', 'Ont.', 'Que.', 'Man.', 'N.S.', 'P.E.I.', 'N.L.',
       'N.B.', 'Sask.', 'Y.T.'], dtype=object)

We can query by any of these 11. (There are no municipalities in Northwest Territories and Nunavut with a population greater than 25,000). 

We can also filter on more than one column, for example, let's try to find all the municipalities in Ontario that have a population greater than 500,000.

In [60]:
df.loc[(df["Prov/terr"] == "Ont.") & (df["Population, 2021"] >= 500000)]

,Name,Prov/terr,"Population, 2021","Population, 2016"
11,Brampton,Ont.,656480,593638
56,Hamilton,Ont.,569353,536917
87,Mississauga,Ont.,717961,721599
106,Ottawa,Ont.,1017449,934243
158,Toronto,Ont.,2794356,2731571


The DataFrame isn't static. We can add or delete columns as needed. Let's first add a column which shows the change in population between 2021 and 2016 and then sort by the cities that lost the most people.

In [66]:
df["Population Change 2021 2016"] = df["Population, 2021"] - df["Population, 2016"]
df.sort_values("Population Change 2021 2016").head(5)

,Name,Prov/terr,"Population, 2021","Population, 2016",Population Change 2021 2016
87,Mississauga,Ont.,717961,721599,-3638
142,Sault Ste. Marie,Ont.,72051,73368,-1317
128,Saguenay,Que.,144723,145949,-1226
157,Timmins,Ont.,41145,41788,-643
22,Cape Breton,N.S.,93694,94285,-591


If we don't want to keep this column, we can just delete it as follows. If you run this cell more than once, i.e. after deleting it, you'll probably get an error saying there is nothing to delete.

In [67]:
del df["Population Change 2021 2016"]

Another particularly useful function in pandas is to group by and summarize data. 

For example, if we want to simply count how many municipalities in each province there are in the dataset we can use the `groupby` and `size` functions.

We use the .reset_index() method to convert the resulting Series back to a DataFrame and give the resulting column a name of 'count'

In [75]:
df.groupby("Prov/terr").size().reset_index(name='count')

,Prov/terr,count
0,Alta.,17
1,B.C.,31
2,Man.,2
3,N.B.,4
4,N.L.,2
5,N.S.,3
6,Ont.,69
7,P.E.I.,1
8,Que.,45
9,Sask.,4


We can chain this to other filters, for example, how many cities in each province have a population greater than 500,000?

In [76]:
df.loc[df["Population, 2021"] >= 500000].groupby("Prov/terr").size().reset_index(name='count')

,Prov/terr,count
0,Alta.,2
1,B.C.,2
2,Man.,1
3,Ont.,5
4,Que.,2


Great! Let's wrap by asking a bit more of an analytical question. 

What's the average percent change in population between 2016 and 2021 by province?

The dataset we'll be working with is [Bike Share ridership](https://open.toronto.ca/dataset/bike-share-toronto-ridership-data/) data from the City of Toronto Open Data portal.

We can download it and save it in a folder as follows:

In [10]:
import urllib.request

year = 2022
url = "https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/7e876c24-177c-4605-9cef-e50dd74c617f/resource/db10a7b1-2702-481c-b7f0-0c67070104bb/download/bikeshare-ridership-" + str(year) + ".zip"
folder = "data"
urllib.request.urlretrieve(url, folder + "/bike-share-ridership-" + str(year) + ".zip")

('data/bike-share-ridership-2022.zip',
 <http.client.HTTPMessage at 0x7fb3b7b03460>)

The zip folder has `.csv` data for each month in the selected year. 

Since our data are zipped, we can either unzip the folder manually and run `df = pd.read_csv(path_to_csv_file)`.

Or we can load using the `zipfile` library. I'm feeding in variables for year and month that can easily allow for switching these out or looping over multiple in the future.

In [16]:
import zipfile

month = '06'

with zipfile.ZipFile("data/bike-share-ridership-" + str(year) + ".zip") as myzip:
    with myzip.open("bikeshare-ridership-" + str(year) + "/Bike share ridership " + str(year) + "-" + month + ".csv") as myfile:
        df = pd.read_csv(myfile)

In [17]:
df

,Trip Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,Bike Id,User Type
0,16028433,384,7430,06/01/2022 00:00,Marilyn Bell Park Tennis Court,7518.0,06/01/2022 00:06,Lake Shore Blvd W / Colborne Lodge Dr,4157,Annual Member
1,16028434,437,7372,06/01/2022 00:00,Adelaide St W / Portland St,7035.0,06/01/2022 00:07,Queen St W / Ossington Ave,1577,Annual Member
2,16028435,495,7156,06/01/2022 00:00,Salem Ave / Bloor St W,7666.0,06/01/2022 00:08,Dundas St W / St Helen Ave - SMART,4628,Casual Member
3,16028436,812,7248,06/01/2022 00:00,Baldwin Ave / Spadina Ave - SMART,7044.0,06/01/2022 00:14,Church St / Alexander St,4137,Annual Member
4,16028437,293,7256,06/01/2022 00:00,Vanauley St / Queen St W - SMART,7416.0,06/01/2022 00:05,Spadina Ave / Blue Jays Way,2295,Annual Member
...,...,...,...,...,...,...,...,...,...,...
605640,16734196,208,7033,06/30/2022 23:59,Union Station,7013.0,07/01/2022 00:02,Scott St / The Esplanade,3895,Casual Member
605641,16734197,1361,7275,06/30/2022 23:59,Queen St W / James St,7422.0,07/01/2022 00:22,Tyndall Ave / King St W - SMART,6971,Casual Member
605642,16734198,1020,7207,06/30/2022 23:59,Dundas St W / Crawford St,7137.0,07/01/2022 00:16,Davenport Rd / Bathurst St,2644,Annual Member
605643,16734199,2177,7076,06/30/2022 23:59,York St / Queens Quay W,7288.0,07/01/2022 00:35,Humber Bay Shores Park West,377,Casual Member
